# Image representation in the frequency domain with Fourier-like transforms

In this short tutorial we'll analyse some properties of the Discrete Fourier Transform (DFT), focusing on the way it represents image pixels in the frequency domain. In particular we'll consider the following three examples:
 * Synthesize an image into its modulus and phase and then reconstruct it from either of these two components.
 * Visualisation of the basis functions associated with the Discrete Cosine Transform (DCT), which is also derived from the DFT.
 * Visualtion of the energy compaction properties of the DCT.
 
Listing and commenting all formal properties of the DFT and DCT, along with their derivation it is outside the scope of the tutorial and the reader is referred to the classical text books on the subject such as:
 * Anil K. Jain, "Fundamentals of digital image processing", Prentice Hall - Pearson, 592 pages, 1988.
 * Jae S. Lim, "Two dimensional: signal and image processing", Prentice Hall, 694 pages, 1990.
 * Rafael Gonzalez and Richard E. Wood, "Digital image processing", 3rd Edition, Pearson, 976 pages, 2007.
 
The aim here is to provide the reader with concrete examples which can be used as basis for further study and elaborations.

## DFT applied over images
We will now apply the 2D DFT over the usual black and white `cameraman` test image. The image is loaded into memory using the `opencv` package and the method `imread`. Over the input image $f\left[m,n\right]$, the following transformation is computed:

$$
\large
F[k,l] = \frac{1}{M\cdot N}\sum_{m=0,n=0}^{M-1,N-1}f[m,n]\cdot e^{-j2\pi\cdot\left(\frac{m\cdot k}{M} + \frac{n\cdot l}{N}\right)}
$$

where $M$ and $N$ denote the image rows and columns, respectively. Representing a given $K$-dimensional signal (an image in this case) using the Fourier analysis allows to highlight the different contributions and select the features which concur to define more the signal's properties. In our case we know that an image can be (roughly) described as an ensemble of edges and colour variations. Literature from psycho-visual studies indicate that edges contribute more to the perception of images by the human visual system (hence the reason why researchers have defined objective quality metrics such as the Structural SIMilarity (SSIM) index). The DFT synthesizes one image into a set of complex numbers (the transform coefficients) each having modulus and phase, whereby the phase contains most of the information associated with the spatial patterns (we'll come back later with further elaborations on this statement). The following Python code cell loads the image into memory and computes the 2D DFT.

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

image = cv2.imread('../input-data/cameraman.tif', cv2.IMREAD_UNCHANGED).astype(np.int32)
image_dft = np.fft.fft2(image)

To start visualising the DFT associated with the image in a convenient way, the modulo and phase can be computed. The modulo in particular will tell us how much each frequency in the 2D frequency space contributes to the signal approximation via (2D) sinusoidal waves. Accordingly, let's plot the modulus of the DFT along side the original gray scale image.

In [ ]:
# Modulus
dft_modulus = np.absolute(np.fft.fftshift(image_dft))
idx = np.where(dft_modulus == 0)
dft_modulus[idx] = 1
dft_log_modulus = np.log(dft_modulus)

# Phase
dft_phase = np.angle(image_dft)

# Plot the results obtained so far
plt.figure(figsize=(15,15))
plt.subplot(1, 2, 1)
plt.imshow(image, cmap='gray')
plt.title('Image: cameraman.tif')
plt.subplot(1, 2, 2)
plt.imshow(dft_log_modulus, cmap='jet')
plt.title('DFT modulus');

From the code above we note that use of `fftshift` when computing the modulo. This function shifts the zero frequency component to the centre of the spectrum: this will make easier to appreciate how the energy is spread along the 2D frequency plane (as exercise try to remove such function from the code above and comment the result). We also note the application of the logarithm to the modulo obtained: this is to *compress* the dynamic of the values without changing their monotonicity so they can be easily visualised. From the plot obtained, we can see how the DFT is able to cluster all the image's energy in the low frequency components (that is around the (0, 0) frequency). There are still some non zero coefficients in the peripheral areas of the 2D plane: this is due to sharp edges present in the picture, most notably between foreground and background.

We now wonder the role of the phase part of the complex value coefficients in characterising an image. To understand this in more detail, we will perform a simple experiment whereby the inverse DFT will be applied over two subsets of the coefficients orginally computed:
 * **Set 1**: Where the phase term is set to zero, so the original image will be reconstructed solely from its modulo.
 * **Set 2**: Where the modulus term is set to the unity (i.e. one), so the original image will be reconstructed solely from its phase.

In [ ]:
rec_modulus = np.fft.ifft2(np.fft.fftshift(dft_modulus))
rec_phase = np.fft.ifft2(np.exp(1j * dft_phase))

plt.figure(figsize=(20,20))
plt.subplot(1, 3, 1)
plt.imshow(image, cmap='gray')
plt.title('Image: cameraman.tif')
plt.subplot(1, 3, 2)
plt.imshow(np.log(np.absolute(rec_modulus)+1), cmap='gray')
plt.title('Image: reconstructed from modulus only')
plt.subplot(1, 3, 3)
plt.imshow(np.absolute(rec_phase), cmap='gray')
plt.title('Image: reconstructed from phase only');

As we may note, most of the morphological information is preserved in the phase only reconstructed image. We've lost some texture and background details but we could still recognise the main content portrayed whilst in the magnitude only reconstructed image we really cannot tell much.

Some additional comments on the code cell above:
 * The modulus part of the transform has been obtained by applying again the function `fftshift` to undo the application of such function during forward transformation
 * To reconstruct the signal from its phase term, we've used the polar representation of complex numbers:
$$
\large
z = a + j\cdot b \Leftrightarrow \rho\cdot e^{j\theta} \text{ where } \rho = \sqrt{a^2 + b^2}, \theta = \arctan\left(\frac{b}{a}\right),
$$
 where in this case $\rho \equiv 1$ and $\theta$ is indeed the phase term of the spectrum computed.
 
We're now in a position to elaborate a bit further on the results obtained, most notably to explain why we stated above: *the phase contains most of the information associated with the spatial patterns*. When reconstructing the image purely from its modulus we set the phase term to zero. This would correspond to assume that all (complex) sinusoidal terms used to compute the DFT have phase equal to zero, that is they are all centred at the origin of the spectrum plan. In that case they would all be symmetrical and only symmetric images can be reconstructed (which is not the case for `cameraman`). Accordingly, we need the relative position of each sinusoidal term (a.k.a. phasor if you're coming from an Electrical Engineering background). And this information is actually contained in the phase term of the DFT coefficients. You can try to repeat the same exercise over a symmetrical image and observed the modulus only reconstructed image. 

## Basis vectors of the 2D 8$\times$8 DCT
We know that the Discrete Cosine Transform (DCT) is defined by extending the input signal to make it even so that the DFT will have real number coefficients only. For a demonstration of the DCT derivation you can have a look at this good [explanation](http://fourier.eng.hmc.edu/e161/lectures/dct/node1.html). Being a real value transform, the DCT will have its basis vectors as real value functions (cosine waves in particular). We are interested here to visualise them for the 2D version of the transform. In particular we want to create the same pictures you can see on the dedicated Wikipedia [page](https://en.wikipedia.org/wiki/Discrete_cosine_transform#Multidimensional_DCTs). These basis can be seen as vectors in an $N$-dimensional space whose combination allows to generate all possible vectors in such $\mathbb{R}^N$ space whereby for an $8\times8$ block, $N = 64$. Each basis is associated with a particular coordinate of such $8\times8$ block, say $(i,j)$. We note that by definition, such basis functions are 2D cosine waves whose 2D spatial frequency is given by:

$$
\large
f_s = \left(\frac{i\pi}{2\sqrt{N}},\frac{j\pi}{2\sqrt{N}}\right).
$$

To generate the basis functions we'll consider 64 different $8\times8$ blocks with all values set to zero except for the one at spatial location $(i,j)$ which is instead set to one. Each one of these blocks can be seen as what would result by the application of the forward DCT to a bidimensional cosine wave vibrating at the 2D spatial frequency associated with $(i,j)$. As such, by applying the inverse DCT, we will indeed obtain that very cosine wave, which, for what said above, corresponds to the basis function associated with $(i,j)$. The example will visualise such basis functions as black and white images where the alternation of pixel values is dictated by the associated 2D frequency.

Accordingly, let's start by importing the required packages in our notebook environment.

In [ ]:
from scipy.fftpack import idct

To compute the inverse DCT we need the `scipy.fftpack` which exports the `idct` function. To generate the basis functions as mentioned above, i.e. by setting to one each element of an $8\times8$ block at the time, we'll write a double for loop whereby for each iteration we will apply the inverse DCT to such block. Moreover, we'll display the obtained bidimensional cosine wave using `imshow` from `matplotlib.pyplot`: this will display a 2D cosine as black and white image. Rather than displaying such images on separated windows, the code below organises them on a $8\times8$ grid: this help also the interpretation of the results since each plot of such grid corresponds to the aforementioned 2D spatial frequency $f_s$.

In [ ]:
plt.figure(figsize=(15,15))
for i in range(8):
    for j in range(8):
        transformed_cosine_wave = np.zeros((8, 8), np.float64)
        transformed_cosine_wave[i, j] = 1
        cosine_wave = idct(idct(transformed_cosine_wave.T, norm='ortho').T, norm='ortho')
        plt.subplot(8, 8, i*8+j+1)
        plt.imshow(cosine_wave, cmap='gray');

To organise the image plots in a grid the code cell above uses the function `subplot` from `matplotlib.pytplot`. Worth also noting from the code the way the inverse DCT is computed. In fact, the DCT is a separable transform, that is for an image it can be computed by applying the 1D DCT over each row and then a 1D DCT over each column. The same also holds for the inverse transformation. For a 2D array, `idct` computes the 1D inverse DCT over the last dimension, i.e. the columns in our case. To start from the rows and compute the transformation correctly, we need to transform the input block: this is done with the `.T` transcript.

From the plot obtained, we note that as we move along (for example) the rows, the alternation of pixel values in each block is concentrated along the vertical lines: this is because when moving along rows we are visiting the vertical frequencies in increasing order. The same observation holds when moving along the columns: this time the frequencies visited are the horizontal ones. Accordingly, a signal containing primarily horizontal edges, will have the DCT coefficients predominantly clustered along the first column (i.e. the one associated with horizontal frequencies).

Finally, as we move along the diagonal of the previous grid of plots, we note check board-like pattern images which denote a 2D cosine wave with horizontal and vertical frequencies given by $f_s$ above.

## Energy compaction of the DCT
It is common knowledge that the DCT when applied over an image will tend to cluster its coefficients primarily on the top-left corner of the transformed image. Such a corner corresponds to the low frequencies which also include the DC coefficient (i.e. the scaled average of all image pixel intensities). The clusterisation described is often referred as the energy compaction property of the DCT and it is key in image compression to provide the codec expert with a convenient representation of the significant frequency components of an image. We'll show in this last example how such a compaction property can be exploited in a practical transform-based image codec.

Accordingly, over the `cameraman` image with $M$ rows and $N$ columns, we consider the DCT applied over image blocks of size $8\times8$, which is the size block size considered in the JPEG compression standard. Practical image codecs do not apply the DCT over the whole image since it would require large computational resources (most notably memory) and more importantly, it is well-known that images may be modelled as space-variant random processes so it is better to decompose their statistics on smaller blocks. Over all $8\times8$ blocks we would then compute the 2D DCT (floating point) and then we'll re-arrange the $8\times8$ coefficients into a 3D array of size $8\times8\times K$ where $K = M\times N / 64$. The following Python code cell performs these operations, i.e. scan of the image pixels over a non-overlapping grid of $8\times8$ blocks, DCT computation and storing of the results in the aforementioned 3D array.

In [ ]:
from scipy.fftpack import dct
M, N = image.shape[0], image.shape[1]
K = (M * N) >> 6

dct_coefficients = np.zeros((8, 8, K))
k = 0

for r in range(M // 8):
    row_sel = slice(r * 8, (r + 1) * 8)
    for c in range(N // 8):
        col_sel = slice(c * 8, (c + 1) * 8)
        block = image[row_sel, col_sel]
        dct_block = dct(dct(block.T, norm='ortho').T, norm='ortho')
        dct_coefficients[:, :, k] = dct_block
        k += 1

We note from the cell above that it is assumed the image having sizes being a multiple of 8. We now have a multitude of $8\times8$ blocks, each with the *energy compaction* fingerprint of the DCT. We are interested to see whether such a energy compaction property can be forwarded at the image level. Accordingly, we can compute the variance of the 3D array built before along the depth dimension (i.e. the $z-$dimension). This would produce a 2D array of size $8\times8$ whereby each element is associated with the variance of a particular DCT subband. The expectation here is that for low-frequency subbands the variance would be higher than their high frequency counterpart (Which, for what we mentioned above, should be close to zero). The following Python cell, compute the variance and shows the resulting 2D array using the `matplotlib` library.

In [ ]:
dct_var = np.var(dct_coefficients, axis=2)
plt.imshow(np.log(dct_var))
plt.xlabel('Vertical frequencies', fontsize=16)
plt.ylabel('Horizontal frequencies', fontsize=16)
plt.title('Variance of DCT subbands (log scale)', fontsize=16)
plt.colorbar();

As expected, the highest value for the variance is associated with the DC coefficient: this is because the average image pixel intensity varies across each $8\times8$ block, most notably over image regions associated with highly textured details. As we move towards the bottom-right corner of the subband 2D plane, the variance gets smaller, confirming the energy compaction property of the DCT whereby all pixels' energy is concentrate in a few coefficients.

## Conclusive remarks
This short tutorial has (hopefully) shown the frequency decomposition operated by Fourier-like transforms over image data. In particular we commented on the modulus and phase of DFT coefficients and considered the case where either of these components is retained, showing which one concurs more to denote the spatial features of an image (e.g. edges). We then showed how the basis vectors of a 2D DCT can be visualised for the classical case of a 64 point transformation. As mentioned in the introduction, it is hoped that the two examples and associated code provided here will help the reader to understand the fundamentals of transform coding for image compression. Finally, we provided a graphical visualisation of the energy compaction property of the 2D DCT whereby most of the image pixels' energy is spread through a few coefficients, primarily concentrated around the low frequency values.